In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,093 kB]
Get:13 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-28 17:27:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.67MB/s    in 0.5s    

2022-08-28 17:27:20 (1.67 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in UFO csv from S3 into a DataFrame
url = "https://ufodatagt.s3.amazonaws.com/Complete_UFO_data+(7).csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("Complete_UFO_data+(7).csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+----------+----------+-----+-------+--------+----------+------------+
|      date|      city|state|country|   shape|  latitude|   longitude|
+----------+----------+-----+-------+--------+----------+------------+
|1949-10-10|san marcos|   tx|     us|cylinder|29.8830556| -97.9411111|
|1956-10-10|      edna|   tx|     us|  circle|28.9783333| -96.6458333|
|1960-10-10|   kaneohe|   hi|     us|   light|21.4180556|-157.8036111|
|1961-10-10|   bristol|   tn|     us|  sphere|    36.595| -82.1888889|
|1965-10-10|   norwalk|   ct|     us|    disk|   41.1175| -73.4083333|
|1966-10-10| pell city|   al|     us|    disk|33.5861111| -86.2861111|
|1966-10-10|  live oak|   fl|     us|    disk|30.2947222| -82.9841667|
|1968-10-10| hawthorne|   ca|     us|  circle|33.9163889|-118.3516667|
|1968-10-10|   brevard|   nc|     us|fireball|35.2333333| -82.7344444|
|1970-10-10|  bellmore|   ny|     us|    disk|40.6686111|    -73.5275|
+----------+----------+-----+-------+--------+----------+------------+
only s

In [5]:
# Create Pandas dataframe from PySpark dataframe
ufo_df = df.toPandas()
ufo_df

,date,city,state,country,shape,latitude,longitude
0,1949-10-10,san marcos,tx,us,cylinder,29.883056,-97.941111
1,1956-10-10,edna,tx,us,circle,28.978333,-96.645833
2,1960-10-10,kaneohe,hi,us,light,21.418056,-157.803611
3,1961-10-10,bristol,tn,us,sphere,36.595000,-82.188889
4,1965-10-10,norwalk,ct,us,disk,41.117500,-73.408333
...,...,...,...,...,...,...,...
172102,2021-02-28,kennewick,wa,us,triangle,46.178823,-119.162978
172103,2021-04-12,kissimmee,fl,us,cylinder,28.255035,-81.429492
172104,2021-03-01,denver,co,us,light,39.745126,-104.959346
172105,2021-03-15,mojave,ca,us,flash,35.097900,-118.130200


In [6]:
from pyspark import SparkFiles
# Load in UFO csv from S3 into a DataFrame
url = "https://ufodatagt.s3.amazonaws.com/supercomputer-power-flops.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("supercomputer-power-flops.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+------+--------+----+------------------------------------+
|Entity|    Code|Year|Floating-Point Operations per Second|
+------+--------+----+------------------------------------+
| World|OWID_WRL|1993|                             1.24E11|
| World|OWID_WRL|1994|                              1.7E11|
| World|OWID_WRL|1995|                              1.7E11|
| World|OWID_WRL|1996|                             3.68E11|
| World|OWID_WRL|1997|                              1.3E12|
| World|OWID_WRL|1998|                              1.3E12|
| World|OWID_WRL|1999|                              2.4E12|
| World|OWID_WRL|2000|                              4.9E12|
| World|OWID_WRL|2001|                              7.2E12|
| World|OWID_WRL|2002|                             3.59E13|
+------+--------+----+------------------------------------+
only showing top 10 rows



In [8]:
# Create Pandas dataframe from PySpark dataframe
comp_df = df.toPandas()
comp_df.head()

,Entity,Code,Year,Floating-Point Operations per Second
0,World,OWID_WRL,1993,1.240000e+11
1,World,OWID_WRL,1994,1.700000e+11
2,World,OWID_WRL,1995,1.700000e+11
3,World,OWID_WRL,1996,3.680000e+11
4,World,OWID_WRL,1997,1.300000e+12


In [9]:
from pyspark import SparkFiles
# Load in UFO csv from S3 into a DataFrame
url = "https://ufodatagt.s3.amazonaws.com/us_population_csv.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("us_population_csv.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+----+---------+
|Year|Total Pop|
+----+---------+
|1900|   77.414|
|1901|   78.904|
|1902|   80.499|
|1903|   82.029|
|1904|   83.622|
|1905|    85.35|
|1906|   87.058|
|1907|   88.677|
|1908|   90.456|
|1909|    92.31|
+----+---------+
only showing top 10 rows



In [10]:
# Create Pandas dataframe from PySpark dataframe
pop_df = df.toPandas()
pop_df.head()

,Year,Total Pop
0,1900,77.414
1,1901,78.904
2,1902,80.499
3,1903,82.029
4,1904,83.622


# Data Cleansing

In [11]:
# Import dependencies
import pandas as pd
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# Check data types
ufo_df.dtypes
# ufo_df['datetime'] = pd.to_datetime(ufo_df['datetime'], format='%Y%m%d')

date          object
city          object
state         object
country       object
shape         object
latitude     float64
longitude    float64
dtype: object

In [13]:
# Change dare data type
ufo_df['date'] = ufo_df['date'].astype('datetime64')

In [14]:
ufo_df.dtypes

date         datetime64[ns]
city                 object
state                object
country              object
shape                object
latitude            float64
longitude           float64
dtype: object

In [15]:
# Extract year from 'date' column
ufo_df['year'] = ufo_df['date'].dt.year 
ufo_df

,date,city,state,country,shape,latitude,longitude,year
0,1949-10-10,san marcos,tx,us,cylinder,29.883056,-97.941111,1949
1,1956-10-10,edna,tx,us,circle,28.978333,-96.645833,1956
2,1960-10-10,kaneohe,hi,us,light,21.418056,-157.803611,1960
3,1961-10-10,bristol,tn,us,sphere,36.595000,-82.188889,1961
4,1965-10-10,norwalk,ct,us,disk,41.117500,-73.408333,1965
...,...,...,...,...,...,...,...,...
172102,2021-02-28,kennewick,wa,us,triangle,46.178823,-119.162978,2021
172103,2021-04-12,kissimmee,fl,us,cylinder,28.255035,-81.429492,2021
172104,2021-03-01,denver,co,us,light,39.745126,-104.959346,2021
172105,2021-03-15,mojave,ca,us,flash,35.097900,-118.130200,2021


In [16]:
# Drop date column and reorder dataframe
del ufo_df['date']

In [17]:
ufo_df = ufo_df[['year', 'city', 'state', 'country', 'shape', 'latitude', 'longitude']]
ufo_df

,year,city,state,country,shape,latitude,longitude
0,1949,san marcos,tx,us,cylinder,29.883056,-97.941111
1,1956,edna,tx,us,circle,28.978333,-96.645833
2,1960,kaneohe,hi,us,light,21.418056,-157.803611
3,1961,bristol,tn,us,sphere,36.595000,-82.188889
4,1965,norwalk,ct,us,disk,41.117500,-73.408333
...,...,...,...,...,...,...,...
172102,2021,kennewick,wa,us,triangle,46.178823,-119.162978
172103,2021,kissimmee,fl,us,cylinder,28.255035,-81.429492
172104,2021,denver,co,us,light,39.745126,-104.959346
172105,2021,mojave,ca,us,flash,35.097900,-118.130200


In [18]:
# Value counts to determine top 5 states
state_count = ufo_df['state'].value_counts()
state_count

ca    21926
fl     9942
wa     9124
tx     8309
ny     7444
      ...  
pe       24
nf       14
yt       14
nl        9
yk        2
Name: state, Length: 68, dtype: int64

In [19]:
# Value counts to determine top 5 countires
country_count = ufo_df['country'].value_counts()
country_count

us    169049
ca      3039
au        10
gb         9
Name: country, dtype: int64

In [20]:
# Value counts to determine top 5 shapes
shape_count = ufo_df['shape'].value_counts()
shape_count

light        37018
circle       17977
triangle     16974
fireball     13307
unknown      12496
other        12231
sphere       11613
disk         10202
oval          7900
formation     5826
changing      4404
cigar         4269
flash         3105
rectangle     2978
cylinder      2784
diamond       2550
chevron       2152
teardrop      1550
egg           1502
cone           676
cross          564
delta           15
round            4
pyramid          2
flare            2
hexagon          2
crescent         2
changed          2
Name: shape, dtype: int64

In [21]:
ufo_df.loc[ufo_df['state'] == 'ca', 'state_california'] = 1
ufo_df.loc[ufo_df['state'] != 'ca', 'state_california'] = 0
ufo_df['state_california'].value_counts()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


0.0    150181
1.0     21926
Name: state_california, dtype: int64

In [22]:
ufo_df.loc[ufo_df['state'] == 'fl', 'state_florida'] = 1
ufo_df.loc[ufo_df['state'] != 'fl', 'state_florida'] = 0
ufo_df['state_florida'].value_counts()

0.0    162165
1.0      9942
Name: state_florida, dtype: int64

In [23]:
ufo_df.loc[ufo_df['state'] == 'wa', 'state_washington'] = 1
ufo_df.loc[ufo_df['state'] != 'wa', 'state_washington'] = 0
ufo_df['state_washington'].value_counts()

0.0    162983
1.0      9124
Name: state_washington, dtype: int64

In [24]:
ufo_df.loc[ufo_df['state'] == 'tx', 'state_texas'] = 1
ufo_df.loc[ufo_df['state'] != 'tx', 'state_texas'] = 0
ufo_df['state_texas'].value_counts()

0.0    163798
1.0      8309
Name: state_texas, dtype: int64

In [25]:
ufo_df.loc[ufo_df['state'] == 'ny', 'state_newyork'] = 1
ufo_df.loc[ufo_df['state'] != 'ny', 'state_newyork'] = 0
ufo_df['state_newyork'].value_counts()

0.0    164663
1.0      7444
Name: state_newyork, dtype: int64

In [26]:
ufo_df.loc[ufo_df['country'] == 'us', 'country_us'] = 1
ufo_df.loc[ufo_df['country'] != 'us', 'country_us'] = 0
ufo_df['country_us'].value_counts()

1.0    169049
0.0      3058
Name: country_us, dtype: int64

In [27]:
ufo_df.loc[ufo_df['country'] == 'ca', 'country_ca'] = 1
ufo_df.loc[ufo_df['country'] != 'ca', 'country_ca'] = 0
ufo_df['country_ca'].value_counts()

0.0    169068
1.0      3039
Name: country_ca, dtype: int64

In [28]:
ufo_df.loc[ufo_df['country'] == 'au', 'country_au'] = 1
ufo_df.loc[ufo_df['country'] != 'au', 'country_au'] = 0
ufo_df['country_au'].value_counts()

0.0    172097
1.0        10
Name: country_au, dtype: int64

In [29]:
ufo_df.loc[ufo_df['country'] == 'gb', 'country_gb'] = 1
ufo_df.loc[ufo_df['country'] != 'gb', 'country_gb'] = 0
ufo_df['country_gb'].value_counts()

0.0    172098
1.0         9
Name: country_gb, dtype: int64

In [30]:
ufo_df.loc[ufo_df['shape'] == 'light', 'shape_light'] = 1
ufo_df.loc[ufo_df['shape'] != 'light', 'shape_light'] = 0
ufo_df['shape_light'].value_counts()

0.0    135089
1.0     37018
Name: shape_light, dtype: int64

In [31]:
ufo_df.loc[ufo_df['shape'] == 'circle', 'shape_circle'] = 1
ufo_df.loc[ufo_df['shape'] != 'circle', 'shape_circle'] = 0
ufo_df['shape_circle'].value_counts()

0.0    154130
1.0     17977
Name: shape_circle, dtype: int64

In [32]:
ufo_df.loc[ufo_df['shape'] == 'triangle', 'shape_triangle'] = 1
ufo_df.loc[ufo_df['shape'] != 'triangle', 'shape_triangle'] = 0
ufo_df['shape_triangle'].value_counts()

0.0    155133
1.0     16974
Name: shape_triangle, dtype: int64

In [33]:
ufo_df.loc[ufo_df['shape'] == 'fireball', 'shape_fireball'] = 1
ufo_df.loc[ufo_df['shape'] != 'fireball', 'shape_fireball'] = 0
ufo_df['shape_fireball'].value_counts()

0.0    158800
1.0     13307
Name: shape_fireball, dtype: int64

In [34]:
ufo_df.loc[ufo_df['shape'] == 'sphere', 'shape_sphere'] = 1
ufo_df.loc[ufo_df['shape'] != 'sphere', 'shape_sphere'] = 0
ufo_df['shape_sphere'].value_counts()

0.0    160494
1.0     11613
Name: shape_sphere, dtype: int64

In [35]:
ufo_df

,year,city,state,country,shape,latitude,longitude,state_california,state_florida,state_washington,...,state_newyork,country_us,country_ca,country_au,country_gb,shape_light,shape_circle,shape_triangle,shape_fireball,shape_sphere
0,1949,san marcos,tx,us,cylinder,29.883056,-97.941111,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1956,edna,tx,us,circle,28.978333,-96.645833,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1960,kaneohe,hi,us,light,21.418056,-157.803611,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1961,bristol,tn,us,sphere,36.595000,-82.188889,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1965,norwalk,ct,us,disk,41.117500,-73.408333,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172102,2021,kennewick,wa,us,triangle,46.178823,-119.162978,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
172103,2021,kissimmee,fl,us,cylinder,28.255035,-81.429492,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172104,2021,denver,co,us,light,39.745126,-104.959346,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
172105,2021,mojave,ca,us,flash,35.097900,-118.130200,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
new_ufo_df = ufo_df[['year', 'state_california', 'state_florida', 'state_washington', 'state_texas', 'state_newyork', 'country_us',
                     'country_ca', 'country_au', 'country_gb', 'shape_light', 'shape_circle', 'shape_triangle', 'shape_fireball', 'shape_sphere']]
new_ufo_df

,year,state_california,state_florida,state_washington,state_texas,state_newyork,country_us,country_ca,country_au,country_gb,shape_light,shape_circle,shape_triangle,shape_fireball,shape_sphere
0,1949,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1956,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1960,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1961,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1965,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172102,2021,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
172103,2021,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
172104,2021,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
172105,2021,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [341]:
# Group by year and sum to get the total number of sightings by state, country, and shape
grouped_df = new_ufo_df.groupby(['year'], as_index = False).sum()

In [38]:
ufo2_df = grouped_df
ufo2_df

,year,state_california,state_florida,state_washington,state_texas,state_newyork,country_us,country_ca,country_au,country_gb,shape_light,shape_circle,shape_triangle,shape_fireball,shape_sphere
0,1910,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1920,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1925,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1929,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1931,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2017,538.0,283.0,182.0,114.0,172.0,4261.0,0.0,0.0,0.0,939.0,599.0,325.0,253.0,320.0
86,2018,294.0,184.0,130.0,77.0,93.0,2724.0,0.0,0.0,0.0,626.0,332.0,221.0,136.0,184.0
87,2019,448.0,307.0,188.0,204.0,156.0,4735.0,0.0,0.0,0.0,1064.0,617.0,433.0,212.0,286.0
88,2020,542.0,318.0,261.0,248.0,227.0,5503.0,0.0,0.0,0.0,1539.0,773.0,371.0,207.0,379.0


In [48]:
comp2_df = comp_df.drop(columns = ['Entity', 'Code'], axis = 1)

,Year,Floating-Point Operations per Second
0,1993,1.240000e+11
1,1994,1.700000e+11
2,1995,1.700000e+11
3,1996,3.680000e+11
4,1997,1.300000e+12


In [52]:
comp3_df = comp2_df.rename(columns={'Year':'year'})
comp3_df.head()

,year,Floating-Point Operations per Second
0,1993,1.240000e+11
1,1994,1.700000e+11
2,1995,1.700000e+11
3,1996,3.680000e+11
4,1997,1.300000e+12


In [64]:
pop2_df = pop_df.rename(columns = {'Year':'year', 'Total Pop': 'Total Population'})
pop2_df.head()

,year,Total Population
0,1900,77.414
1,1901,78.904
2,1902,80.499
3,1903,82.029
4,1904,83.622


In [65]:
# Merge groupby dataframe and computing power dataframe
ufo = ufo2_df.merge(comp3_df, how = 'inner', on = 'year')

In [66]:
# Merge above dataframe with total population dataframe
ufo_final = ufo.merge(pop2_df, how = 'inner', on = 'year')
ufo_final.head()

,year,state_california,state_florida,state_washington,state_texas,state_newyork,country_us,country_ca,country_au,country_gb,shape_light,shape_circle,shape_triangle,shape_fireball,shape_sphere,Floating-Point Operations per Second,Total Population
0,1993,64.0,12.0,31.0,33.0,35.0,506.0,10.0,0.0,0.0,66.0,39.0,111.0,12.0,35.0,1.240000e+11,264.300568
1,1994,110.0,32.0,30.0,23.0,49.0,699.0,16.0,0.0,0.0,95.0,56.0,139.0,31.0,54.0,1.700000e+11,266.915822
2,1995,99.0,32.0,73.0,52.0,51.0,895.0,18.0,0.0,0.0,164.0,63.0,159.0,40.0,43.0,1.700000e+11,269.466555
3,1996,128.0,45.0,60.0,56.0,31.0,949.0,29.0,1.0,0.0,197.0,54.0,175.0,36.0,56.0,3.680000e+11,271.967753
4,1997,244.0,84.0,137.0,123.0,92.0,2087.0,37.0,0.0,0.0,467.0,117.0,348.0,75.0,135.0,1.300000e+12,274.617188


In [184]:
# Reorder and select columns to create dataframe for state linear regression
ml_state_df = ufo_final[['year', 'Floating-Point Operations per Second', 'Total Population','state_california', 'state_florida', 'state_washington', 'state_texas', 'state_newyork']]
ml_state_df.head(10)

,year,Floating-Point Operations per Second,Total Population,state_california,state_florida,state_washington,state_texas,state_newyork
0,1993,1.240000e+11,264.300568,64.0,12.0,31.0,33.0,35.0
1,1994,1.700000e+11,266.915822,110.0,32.0,30.0,23.0,49.0
2,1995,1.700000e+11,269.466555,99.0,32.0,73.0,52.0,51.0
3,1996,3.680000e+11,271.967753,128.0,45.0,60.0,56.0,31.0
4,1997,1.300000e+12,274.617188,244.0,84.0,137.0,123.0,92.0
5,1998,1.300000e+12,277.167959,380.0,168.0,235.0,166.0,120.0
6,1999,2.400000e+12,279.702500,660.0,166.0,382.0,242.0,171.0
7,2000,4.900000e+12,289.550687,778.0,207.0,339.0,246.0,200.0
8,2001,7.200000e+12,292.472818,672.0,249.0,387.0,234.0,259.0
9,2002,3.590000e+13,295.227229,755.0,269.0,372.0,312.0,257.0


In [197]:
# Reorder and select columns to create dataframe for shape linear regression
ml_shape_df = ufo_final[['year', 'Floating-Point Operations per Second', 'Total Population', 'shape_light', 'shape_circle', 'shape_triangle', 'shape_fireball', 'shape_sphere']]
ml_shape_df.head()

,year,Floating-Point Operations per Second,Total Population,shape_light,shape_circle,shape_triangle,shape_fireball,shape_sphere
0,1993,1.240000e+11,264.300568,66.0,39.0,111.0,12.0,35.0
1,1994,1.700000e+11,266.915822,95.0,56.0,139.0,31.0,54.0
2,1995,1.700000e+11,269.466555,164.0,63.0,159.0,40.0,43.0
3,1996,3.680000e+11,271.967753,197.0,54.0,175.0,36.0,56.0
4,1997,1.300000e+12,274.617188,467.0,117.0,348.0,75.0,135.0


### Linear Regression- State

In [311]:
X = ml_state_df.drop(columns = ['year', 'Floating-Point Operations per Second', 'Total Population'])
y = ml_state_df['Floating-Point Operations per Second']

In [312]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [313]:
model = LinearRegression()

In [314]:
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

In [315]:
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.49931438957944974
Testing Score: 0.479300787596981


In [316]:
X = ml_state_df.drop(columns = ['year', 'Floating-Point Operations per Second', 'Total Population'])
y = ml_state_df['Total Population']

In [317]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [318]:
model = LinearRegression()

In [319]:
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

In [320]:
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.7999477781997839
Testing Score: 0.7022625127649019


Population is a better predictor for number of ufo sightings by top 5 states

## Linear Regression- Shape

In [321]:
X = ml_shape_df.drop(columns = ['year', 'Floating-Point Operations per Second', 'Total Population'])
y = ml_shape_df['Floating-Point Operations per Second']

In [322]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [323]:
model = LinearRegression()

In [324]:
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

In [325]:
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.5828972297452435
Testing Score: 0.6723638929235534


In [326]:
X = ml_shape_df.drop(columns = ['year', 'Floating-Point Operations per Second', 'Total Population'])
y = ml_shape_df['Total Population']

In [327]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [328]:
model = LinearRegression()

In [329]:
model.fit(X_train, y_train)
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

In [330]:
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.8465213818010165
Testing Score: 0.7768097926971572


Total Population is a better predictor for number of ufo sightings by shape

# Predictions

In [331]:
# Define a function to feed the predictions dataframe
def ufo_data(x,y,growth,year):
  ufo_y =[]
  ufo_x =[]
  x = x[year:]
  y = y[year:]
  slope, intercept, rvalue, pvalue, stderr = linregress(x,y)
  start_x = x[-1]
  start_y = y[-1]
  for i in range(2100 - start_x):
    i += 1
    slope = slope * growth
    new_x = start_x + i
    new_y = slope*(new_x) + intercept
    ufo_y.append(new_y)
    ufo_x.append(new_x)
  return(ufo_x,ufo_y)


In [332]:
# Define a function to create the predictions dataframe
def ufo_future(df,growth,year):
  columns = df.columns.to_list()
  new_df = pd.DataFrame(columns=columns)
  for i in columns:
      if (i == 'year'):
        x = df.year.to_list()
      else:
        y = df[f'{i}'].to_list()
        new_x, new_y = ufo_data(x,y,growth,year)
        new_df[f'{i}']=new_y
        new_df['year']=new_x
  return new_df

In [333]:
state_growth = ufo_future(ml_state_df, 1.0001, -50)

In [334]:
state_growth

,year,Floating-Point Operations per Second,Total Population,state_california,state_florida,state_washington,state_texas,state_newyork
0,2018,4.605572e+16,341.036437,1251.740611,664.439533,493.115418,433.161411,424.453965
1,2019,4.907454e+16,344.655084,1296.120121,693.921390,508.753701,447.048443,440.560111
2,2020,5.209391e+16,348.274393,1340.507762,723.408648,524.394848,460.938019,456.669207
3,2021,5.511383e+16,351.894365,1384.903535,752.901308,540.038862,474.830139,472.781255
4,2022,5.813431e+16,355.515001,1429.307441,782.399371,555.685741,488.724805,488.896255
...,...,...,...,...,...,...,...,...
78,2096,2.831907e+17,625.289105,4737.849257,2980.304570,1721.537101,1524.018459,1689.627340
79,2097,2.862529e+17,628.959754,4782.866529,3010.210100,1737.400116,1538.105057,1705.964942
80,2098,2.893156e+17,632.631073,4827.892025,3040.121093,1753.266029,1552.194227,1722.305528
81,2099,2.923789e+17,636.303062,4872.925745,3070.037550,1769.134840,1566.285971,1738.649099


In [337]:
state_growth.to_csv('state_sighting_predictions.csv')

In [338]:
shape_growth = ufo_future(ml_shape_df, 1.0001, -50)

In [339]:
shape_growth

,year,Floating-Point Operations per Second,Total Population,shape_light,shape_circle,shape_triangle,shape_fireball,shape_sphere
0,2018,4.605572e+16,341.036437,2300.120553,1216.334256,904.465153,998.279757,738.488442
1,2019,4.907454e+16,344.655084,2392.747304,1272.291069,933.384834,1044.143299,769.810315
2,2020,5.209391e+16,348.274393,2485.391025,1328.258135,962.309813,1090.015245,801.137925
3,2021,5.511383e+16,351.894365,2578.051719,1384.235454,991.240092,1135.895594,832.471275
4,2022,5.813431e+16,355.515001,2670.729388,1440.223028,1020.175671,1181.784348,863.810366
...,...,...,...,...,...,...,...,...
78,2096,2.831907e+17,625.289105,9576.156676,5611.865706,3176.170053,4600.962446,3198.890767
79,2097,2.862529e+17,628.959754,9670.114534,5668.626655,3205.505329,4647.485078,3230.662755
80,2098,2.893156e+17,632.631073,9764.089555,5725.397973,3234.845963,4694.016207,3262.440546
81,2099,2.923789e+17,636.303062,9858.081742,5782.179662,3264.191957,4740.555837,3294.224143


In [340]:
shape_growth.to_csv('shape_sighting_predictions.csv')

## Totals

In [168]:
total = ufo2_df[['state_california', 'state_florida', 'state_washington', 'state_texas', 'state_newyork']].sum()
total

state_california    21926.0
state_florida        9942.0
state_washington     9124.0
state_texas          8309.0
state_newyork        7444.0
dtype: float64

In [169]:
total_shapes = ufo2_df[['shape_light', 'shape_circle', 'shape_triangle', 'shape_fireball', 'shape_sphere']].sum()
total_shapes

shape_light       37018.0
shape_circle      17977.0
shape_triangle    16974.0
shape_fireball    13307.0
shape_sphere      11613.0
dtype: float64